In [12]:
"""
Author: Pengbo Li
Created: 2024-10-22
Version: 1.0

Description: 
    Simulation of the Aluminum cylinder LVDT design.
    Check the LVDT performance of the Aluminum cylinder, magnet core, and the combination of both.

Model:  
- Aluminum cylinder, including change the inner diameter, outer diameter, and length.
- magnet core, including change the diameter and length.
- air coil + magnet core, including different combinations of the magnet core and air coil.

"""

import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from modules import simulator, geometry, coreConfig, dataHandler

def run_lvdt_simulation(path, filename, simulation_freq, simulation_amplitude, core_params, coil_params):
    """
    Run the LVDT simulation with specified parameters and save the results to a file.

    Parameters:
        path (str): Path to save the data file.
        filename (str): Name of the data file.
        simulation_freq (float): Simulation frequency in Hz.
        simulation_amplitude (float): Amplitude for the outer coil circuit.
        core_params (tuple): Tuple containing (outer_diameter, inner_diameter) for the magnet geometry.
        coil_params (dict): Dictionary containing parameters for CoreCoil, MiddleCoil, and OuterCoil.
    """
    # Define geometry for core and coils
    Magnet_geo = geometry.def_core_geo(*core_params['Magnetcore'])
    Alu_geo  = geometry.def_cylinder_geo(*core_params['AluminumCylinder'])
    MiddleCoil_geo = geometry.def_coil_geo(*coil_params['MiddleCoil'])
    OuterCoil_geo  = geometry.def_coil_geo(*coil_params['OuterCoil'])
    # Define circuit properties
    MC_circuit = simulator.def_circuit_prop("middlecoil", 0, 0)
    OC_upper_circuit = simulator.def_circuit_prop("outercoil_upper", simulation_freq, simulation_amplitude)
    OC_lower_circuit = simulator.def_circuit_prop("outercoil_lower", simulation_freq, -simulation_amplitude)

    # Set up simulation environment
    simulator.def_femm_problem(signal_frequency=simulation_freq)
    simulator.build_air_geometry("Outside", 10)

    # Build geometry and assign labels
    m_label = simulator.build_core_geometry(Magnet_geo, 1)
    alu_label = simulator.build_cylinder_geometry(Alu_geo, 2)
    mc_label = simulator.build_coil_geometry(MiddleCoil_geo, MC_circuit, 3, customized_material=True)
    oc_upper_label = simulator.build_coil_geometry(OuterCoil_geo, OC_upper_circuit, 4, customized_material=True)
    oc_lower_label = simulator.build_coil_geometry(OuterCoil_geo, OC_lower_circuit, 5, customized_material=True, reverse=True)

    print("Magnet label:", m_label)
    print("Aluminum label:", alu_label)
    print("Middle coil label:", mc_label)
    print("Outer coil upper label:", oc_upper_label)
    print("Outer coil lower label:", oc_lower_label)

    # Simulation configuration
    config = coreConfig.moving_config(-5, 1, 10)
    lvdt_data = simulator.def_lvdt_data(config['steps'])

    # Run the simulation
    sim_results = simulator.lvdt_simulation(
        moving_parts_label=[1, 2],
        CC_config=config,
        lvdt_data=lvdt_data,
        OC_upper_circuit=OC_upper_circuit,
        OC_lower_circuit=OC_lower_circuit,
        MC_circuit=MC_circuit,
        CC_circuit=None
    )

    # Save results
    dataHandler.save_data(sim_results, path + filename)
    print("Data saved to:", path + filename)

In [13]:
filepath = "../data/"

simulation_freq = 10000 
simulation_amplitude = 0.02  

# Define parameters for cores
# For the Magnetcore, the parameters are (Diameter, Length, Material)
# For the AluminumCylinder, the parameters are (Inner_Diameter, Outer_Diameter, Length, Material)
core_params ={
    'MagnetCore': (4, 10, "N40"),
    'AluminumCylinder': (6, 8, 12, 'Aluminum, 6061-T6')
}
# Define parameters for coils
# For the coil, the parameters are ( Wire_Material, Wire_Diameter, Wire_Insulation_thickness, Layers, 
#                                      Coil_Length, Coil_Inner_Diameter, Coil_Distance)
coil_params = {
    'MiddleCoil': ("100um", 0.1, 0, 16, 3, 18, 0),
    'OuterCoil': ("100um", 0.1, 0, 16, 3, 18, 16)
}

filename_template = "LVDT_alucyld_id{alu_inn_diam:02d}_od{alu_out_diam:02d}_l{alu_len}.h5"
alu_cyld_inn_diam = [6, 7, 8, 9]
alu_cyld_out_diam = [14]
alu_iter = coreConfig.params_iter(alu_cyld_inn_diam, alu_cyld_out_diam)

for iter in alu_iter:
    core_params['AluminumCylinder'] = (iter[0], iter[1], 12, 'Aluminum, 6061-T6')
    filename = filename_template.format(inn_diam=iter[0],out_diam=iter[1],  len=12)
    print(filename)
    # run_lvdt_simulation(filepath, filename, simulation_freq, simulation_amplitude, core_params, coil_params)



LVDT_alucyld_id06_od14_l12.h5
LVDT_alucyld_id07_od14_l12.h5
LVDT_alucyld_id08_od14_l12.h5
LVDT_alucyld_id09_od14_l12.h5


In [ ]:
from modules import jsonHandler as js
path = "../data/"
filename, simulation_freq, simulation_amp, core_params, coil_params = js.load_parameters_from_json('../lvdt_parameters.json')

run_lvdt_simulation(
    path=path,
    filename=filename,
    simulation_freq=simulation_freq,
    simulation_amplitude=simulation_amp,
    core_params=core_params,
    coil_params=coil_params
)